In [10]:
import pandas as pd
import os

In [11]:
dow_jones_path = os.path.join('Resources', 'DowJones.csv')
nasdaq_path = os.path.join('Resources', 'Nasdaq.csv')
sp500_path = os.path.join('Resources', 'SP500.csv')

In [15]:
dow_jones_df = pd.read_csv(dow_jones_path)
nasdaq_df = pd.read_csv(nasdaq_path)
sp500_df = pd.read_csv(sp500_path)

In [19]:
dow_jones_df.Date = pd.to_datetime(dow_jones_df.Date)
nasdaq_df.Date = pd.to_datetime(nasdaq_df.Date)
sp500_df.Date = pd.to_datetime(sp500_df.Date)

In [20]:
dow_jones_df = dow_jones_df.rename(columns = {'Value': 'Dow_Jones'})
nasdaq_df = nasdaq_df.rename(columns = {'Value': 'NASDAQ'})
sp500_df = sp500_df.rename(columns = {'Value': 'S&P_500'})

In [21]:
combined_df = sp500_df.merge(nasdaq_df, how = 'outer', on = 'Date')\
              .merge(dow_jones_df, how = 'outer', on = 'Date')

In [22]:
combined_df[combined_df.isna().any(axis = 1)]

,Date,S&P_500,NASDAQ,Dow_Jones
1151,1999-09-24,1277.36,NaN,NaN
1254,2001-09-11,1092.54,1695.37,NaN
1346,2003-06-19,994.70,NaN,NaN
1388,2004-04-08,1139.32,2052.88,NaN
1649,2009-04-09,856.56,NaN,NaN
2084,1999-09-23,NaN,2749.83,10318.59
2085,2003-06-20,NaN,1644.72,9200.75
2086,2001-09-10,NaN,NaN,9605.51


In [23]:
combined_df.Date.value_counts()

1977-09-02    1
2004-07-16    1
2004-07-02    1
2004-06-25    1
2004-06-18    1
             ..
1990-11-30    1
1990-11-23    1
1990-11-16    1
1990-11-09    1
2001-09-10    1
Name: Date, Length: 2087, dtype: int64

In [24]:
path = os.path.join('Resources', 'joined_stocks.csv')
combined_df.to_csv(path, index = False)